In [1]:
#Hard

In [1]:
#Start finbert
from transformers import BertTokenizer, BertForSequenceClassification
from transformers import pipeline

finbert = BertForSequenceClassification.from_pretrained('yiyanghkust/finbert-tone',num_labels=3)
tokenizer_finbert = BertTokenizer.from_pretrained('yiyanghkust/finbert-tone')

nlp = pipeline("sentiment-analysis", model=finbert, tokenizer=tokenizer_finbert)

In [2]:
#Start FinGPT
from transformers import AutoModel, AutoTokenizer, AutoModelForCausalLM, LlamaForCausalLM, LlamaTokenizerFast
from peft import PeftModel  # 0.5.0

base_model = "NousResearch/Llama-2-13b-hf" 
peft_model = "FinGPT/fingpt-sentiment_llama2-13b_lora"
tokenizer = LlamaTokenizerFast.from_pretrained(base_model, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
model = LlamaForCausalLM.from_pretrained(base_model, trust_remote_code=True, device_map = "cuda:0", load_in_8bit = True,)
model = PeftModel.from_pretrained(model, peft_model)
model = model.eval()

ImportError: 
The new behaviour of LlamaTokenizer (with `self.legacy = False`) requires the protobuf library but it was not found in your environment. Checkout the instructions on the
installation page of its repo: https://github.com/protocolbuffers/protobuf/tree/master/python#installation and follow the ones
that match your environment. Please note that you may need to restart your runtime after installation.


In [3]:
#Start Other 
import translators as ts
def translate(message):
    try:
        message = ts.translate_text(message, translator='google', from_language='ru', to_language='en')
        return (True, message)
    except Exception:
        return (False, message)

In [4]:
#finbert func
def finbert(text):
    state, text = translate(text)
    if state == 0:
        return 0
        
    sentences = [text]
    label = {
        "Negative": 1,
        "Neutral": 3,
        "Positive" : 5
        
    }
    results = nlp(sentences)
    return label[results[0]['label']]

In [5]:
#FinGPT func
def fingpt(text):
    state, text = translate(text)
    text = '''Instruction: What is the sentiment of this news? Please choose an answer from {negative/neutral/positive}
        Input: '''+text+'''
        Answer: '''
    tokens = tokenizer(text, return_tensors='pt', padding='max_length', max_length=512)
    res = model.generate(**tokens, max_length=512)
    res_sentences = [tokenizer.decode(i) for i in res]
    out_text = [o.split("Answer: ")[1] for o in res_sentences]
    out_text = out_text[0]
    if out_text.find("negative") != -1:
        res = 1
    elif out_text.find("positive") != -1:
        res = 5
    elif out_text.find("neutral") != -1:
        res = 3
    else:
        res = 0

    return res



In [6]:
#Random func
def random_gpt(text):
    pass

In [7]:
def check_news(news):
    res_1 = fingpt(news)
    res_2 = finbert(news)
    res_3 = random_gpt(news)
    #res = (res_1+res_2)/2 #Бред, там есть 0
    print(res_1, res_2)

In [8]:
check_news("Газпром вырос на 15%")

NameError: name 'tokenizer' is not defined